📊 Stats to Explore for GW1
1. 🔥 Top Points Per Game(PPG) Players from 2024/25
2. 💰 Best Value Players (Points per Million)

In [ ]:
import json
import pandas as pd

with open('../data/bootstrap_static.json') as f:
    data = json.load(f)

players_df = pd.DataFrame(data['elements'])
teams_df = pd.DataFrame(data['teams'])

# Example: Top 15 players by most points per game
most_ppg = players_df[['web_name', 'points_per_game', 'now_cost', 'minutes']].sort_values(by='points_per_game', ascending=False).head(15)

# Example: Top 15 players by points per million
players_df['points_per_million'] = players_df['total_points'] / (players_df['now_cost'] / 10)

points_per_million = players_df[['web_name', 'total_points', 'now_cost', 'points_per_million']].sort_values(by='points_per_million', ascending=False).head(15)

# Example: Top 10 defenders by total points
# Note: 'element_type' 2 corresponds to defenders in the FPL data
defenders = players_df[players_df['element_type'] == 2] 
top_defenders = defenders[['web_name', 'total_points', 'clean_sheets', 'goals_scored', 'assists', 'bonus', 'now_cost']].sort_values(by='total_points', ascending=False).head(10)

# Example: Players with injury updates and their chance of playing this round
flagged = players_df[players_df['news'].str.len() > 0]
flagged[['web_name', 'news', 'chance_of_playing_this_round']]



,web_name,news,chance_of_playing_this_round
30,G.Jesus,Knee injury - Unknown return date,None
33,Martinez,Suspended until 23 Aug,None
36,Marschall,Has joined Stevenage permanently.,None
57,Barrenechea,Season-long loan to Benfica,None
126,Cook,Suspended until 30 Aug,None
136,Enes Ünal,Knee injury - Unknown return date,None
178,Rushworth,Has joined Coventry City on loan for the rest ...,None
179,Scherpen,Has joined Union SG permanently,None
184,Estupiñan,Has joined AC Milan permanently,None
191,Cashin,Has joined Birmingham on loan for the rest of ...,None
